# Unigram Mixture model


##### The model :
First, we sample the topic of a document as a one-hot vector, from a multinomial of size $K$.
- $z \sim \mathcal{M}(1, (\pi_1, \dots, \pi_K)), z\in\{0, 1\}^K$
- $p(z)=\prod_{k=1}^K\pi_k^{z_k}$

Once the topic $z^i$ is selected for document $i$, we sample the documents $N$ words from a multinomial of size $d$, the vocabulary size.
- $w^i_n~|~\{z^i_k = 1\} \sim \mathcal{M}(1, (b_{1k},\dots,b_{dk}))$

In plate notations :
<img src="img/unigram_mixture_new.png" alt="unigram mixture" width="200"/>

Finally :
$\displaystyle{p(w^i, z^i) = \prod_{j=1}^d\prod_{k=1}^K \prod_{n=1}^{N}(b_{jk}\pi_k)^{w^i_n(j)z_k}}$


### Question 0:
Our first goal will be to sample documents according to this generating process. What is an alternative to sampling $N$ multinomials of size $1$ ? Write the new joint probability distribution and draw the new model in plate notation.

### Question 1:
Sample $M=100$ documents of $N=30$ words from the given $\pi$ and $b$ in the next cell. We have $d=3$ words, but $K=6$ topics.
Write a function _gen_\__corpus_ that takes all the required parameters as input, and returns a (M,d) array containg the $x^{(i)}$, as well as a (M,) array containing the topics associated with these documents.

In [ ]:
import numpy as np

n_docs = 400
doc_length = 30
n_topics = 6
n_tokens = 3

topic_proba = np.array([1./n_topics] * n_topics)
word_proba = np.array([
    [0.8, 0.1, 0.1],
    [0.1, 0.8, 0.1],
    [0.1, 0.1, 0.8],
    [0.45, 0.45, 0.1],
    [0.1, 0.45, 0.45],
    [0.45, 0.1, 0.45]
])


def gen_corpus(n_docs, n_topics, n_tokens, doc_length, topic_proba, word_proba):
    """
    n_docs: number of documents, S
    n_topics: number of topics, K
    doc_length: number of words per documents, N
    topic_proba: (K,) array containing the pi_k
    word_proba: (K, d) array containing the b_{k,j}.
    """
    corpus = np.empty((n_docs, n_tokens), dtype='int32')
    topics = # sample the topic of each documents

    # Select corresponding multinomials
    for i_doc, topic in enumerate(topics):
        pass  # Sample the count of words in each documents
    return corpus, topics

corpus, topics = gen_corpus(n_docs, n_topics, n_tokens, doc_length, topic_proba, word_proba)

### Visualizing the data :
In the next cell, we represent this corpus. We will use the three vertices of a triangle to represent our three tokens. A document will be represented as a convex combination of the three vertices. We will color each document according to their topic (red, green, blue and black).
Finally, each row in $b$ giving the token probability given the topic will also be visualized.

### Question 2:
Run the next cell to visualize the dataset. Explain what is shown in the figure produced.

In [ ]:
%matplotlib inline
import math

from matplotlib import pyplot as plt
from matplotlib.patches import Polygon
from matplotlib.collections import PatchCollection

# Compute the triangle vertices using complex roots of 1
vertex_ids = np.array(range(n_tokens))
vertices = np.vstack([
    np.cos(2 * math.pi * vertex_ids / n_tokens),
    np.sin(2 * math.pi * vertex_ids / n_tokens)]
).T

# Plot the triangle
fig, ax = plt.subplots(figsize=(10, 10))
polygon = Polygon(vertices, fill=False, linewidth=2, linestyle='--')
ax.add_patch(polygon)
ax.set_xlim([-1.1, 1.1])
ax.set_ylim([-1.1, 1.1])

# Compute a S x 2 array containing documents as linear combination of our vertices

linear_combinations = (corpus / np.sum(corpus, axis=1)[:, None]) @ vertices  
colors = np.array([[1, 0, 0], [0, 1, 0], [0, 0, 1], [0, 0, 0], [1, 1, 0], [0, 1, 1]])

plt.scatter(
    linear_combinations[:, 0], linear_combinations[:, 1], c=np.array(topics) @ colors,
    marker='o', s=500, alpha=0.6
)

centers = word_proba @ vertices

plt.scatter(
    centers[:, 0], centers[:, 1], c=colors,
    marker='X', s=500, alpha=1, edgecolor=(0, 0, 0, 1), linewidths=2
)

plt.axis('off')

plt.show()

We observe documents $(x^{(i)})_{i=1..M}$, and apply EM for $t=1\dots T$.


##### E-Step:
- $\displaystyle{p(z_k=1~|~x^{(i)};b^{(t-1)};\pi^{(t-1)}) = \frac{\pi^{(t-1)}_k \prod_{j=1}^d(b_{jk}^{(t-1)})^{x_j}}{\sum_{k'=1}^K\pi^{(t-1)}_{k'}\prod_{j=1}^d(b_{jk'}^{(t-1)})^{x_j}}}$
- $q^{(t)}_{ik} = \mathbb{E}[z_k~|~x^{(i)};b^{(t)};\pi^{(t)}]$


##### M-Step:
- $\displaystyle{b_{jk}^{(t)} = \frac{\sum_{i}x_j^{(i)}q_{ik}^{(t)}}{\sum_{i,j'}x_{j'}^{(i)}q_{ik}^{(t)}}}$ and $\displaystyle{\pi_{k}^{(t)} = \frac{\sum_{i}q_{ik}^{(t)}}{\sum_{i,k'}q_{ik'}^{(t)}}}$


### Question 3:
1. Show the formula for $b^{(t)}_{jk}$ starting from the expected complete log-likelihood :
$$\mathbb{E}_{q_i^{(t)}}[\log(p(X, Z;b,\pi)]=\sum_{i,j,k}x_j^{(i)}q_{ik}^{(t)}\log(b_{jk}) + \sum_{i,k}q_{ik}^{(t)}\log(\pi_k)$$
2. What problem can happen when computing the $q_{i,k}$ in the E step ?
3. Write code for logsumexp which computes $\log(\sum_i\exp(x_i))$, beware of (2.). Write a test that shows the implementation works as expected. How can we use this function in the E step ?
4. Fill in the code for the E and M steps.

In [ ]:
def logsumexp(x):
    """
    x: a (n, m) array.
    return: log(sum(exp(x_i,:))) for all i, avoiding numerical underflow
    """
    pass

# test that our function does what it should on simple inputs
x = np.array([[1, 2, 3], [2, 2, 2]]) 
print(logsumexp(x), np.log(np.sum(np.exp(x), axis=1)))

# write a test case in which the basic method overflows and yours doesn't
x = np.array([[...], [...]]) 
print(logsumexp(x), np.log(np.sum(np.exp(x), axis=1))) # 


def E_step(corpus, learned_topic_proba, learned_word_proba):
    """
    corpus : (S, n_tokens) array containing the counts of words in each document
    learned_topic_proba : (n_topics, ) array containing the probability of each topic
    learned_topic_proba : (n_topics, n_token) array containing the probability of each word for each topic
    returns : doc_topic_proba : (S, n_topicdlhkfiidgninkhvulrjvurlvhrdekcur) the q_ik
    """
    pass

def M_step(corpus, doc_topic_probas):
    """
    corpus : (S, n_tokens) array containing the counts of words in each document
    doc_topic_probas : (S, n_topics) array containing q_ik computed in the E-step
    """
    pass


### Log-Likelihood
It is convenient to visualize the log-likelihood of our data during training to make sure it is being maximized.

### Question 4:
1. Implement computation of the log-likelihood in the next cell. <em>log_likelihood_qik</em> will be used later in the session.

In [ ]:
def log_likelihood_E_step(corpus, doc_topic_probas, learned_word_proba, learned_topic_proba):
    """
    corpus : (S, n_tokens) array containing the counts of words in each document
    doc_topic_probas : (S, n_topics) array containing E_q[z] computed in the E-step
    learned_topic_proba : (n_topics, ) array containing the probability of each topic
    learned_word_proba : (n_topics, n_token) array containing the probability of each word for each topic
    """
    first_term = corpus @ np.log(learned_word_proba.T)
    first_term[np.isnan(first_term)] = 0
    first_term *= doc_topic_probas
    
    second_term = doc_topic_probas @ np.log(learned_topic_proba)[:, None]
    entropy = np.log(doc_topic_probas) * doc_topic_probas
    entropy[np.isnan(entropy)] = 0
    return (np.sum(first_term) + np.sum(second_term) - np.sum(entropy)) / corpus.shape[0]

def log_likelihood(corpus, learned_word_proba, learned_topic_proba):
    """
    corpus : (S, n_tokens) array containing the counts of words in each document
    learned_topic_proba : (n_topics, ) array containing the probability of each topic
    learned_word_proba : (n_topics, n_token) array containing the probability of each word for each topic
    """
    pass

### Running EM :
We can now run the Expectation-Maximization algorithm on our corpus.
We will initialize the algorithm by setting $\pi_k = 1/K$.
For $b_{k:}$, we use the proportions of random documents.
Then we run the algorithm for a few iterations.

At completion, we plot the log likelihood curve for train and valid.

### Question 5:
- Complete the following code to run EM, plot the train and valid log-likelihood. Also plot the log-likelihood obtain from the true parameters <em>topic_proba</em> and <em>word_proba</em>. Comment.
- Compute the train log-likelihood after the E-step. Compare the curve of this log-likelihood and train_ll_E_step. Comment.

In [ ]:

valid_corpus, valid_topics = gen_corpus(1000, n_topics, n_tokens, doc_length, topic_proba, word_proba)

n_iter = 20
n_learned_topics = 6

# Initialize the learned parameter
# uniform topic proba
learned_topic_proba = np.ones((n_learned_topics,)) / n_learned_topics

# word proba taken from random documents + some constant to avoid zeroes
learned_word_proba = corpus[np.random.permutation(n_docs)[:n_learned_topics]] + 1e-3
learned_word_proba = learned_word_proba / np.sum(learned_word_proba, axis=1)[:, None]

plt.figure(dpi=150)
train_ll = []
valid_ll = []
for i in range(n_iter):
    # step of the EM algorithm
    # compute the log-likelihoods and append to train_ll, valid_ll
    
# plot train_ll, valid_ll

# compute valid log-likelihood for real parameters of model and plot
    
plt.xlabel("EM-iteration")
plt.ylabel("Data log-likelihood")
plt.legend(bbox_to_anchor=(1,1), loc=2)
plt.show()


### Question 6:
Plot the learned word_proba along the dataset and original word_probas. Repeat the process of learning / visualizing, comment.

In [ ]:
# Plot the triangle
fig, ax = plt.subplots(figsize=(10, 10))
polygon = Polygon(vertices, fill=False, linewidth=2, linestyle='--')
ax.add_patch(polygon)
ax.set_xlim([-1.1, 1.1])
ax.set_ylim([-1.1, 1.1])

# Compute a S x 2 array containing documents as linear combination of our vertices
linear_combinations = (corpus / np.sum(corpus, axis=1)[:, None]) @ vertices  

plt.scatter(
    linear_combinations[:, 0], linear_combinations[:, 1], c=np.array(topics) @ colors,
    marker='o', s=500, alpha=0.2
)

# plot real word_proba

centers = word_proba @ vertices

plt.scatter(
    centers[:, 0], centers[:, 1], c=colors,
    marker='X', s=500, alpha=1, edgecolor=(0, 0, 0, 1), linewidths=2
)

# plot the learned word_proba.

plt.show()

### Larger synthetic dataset

Now, we will generate a more realistic synthetic dataset. We will have $1000$ documents of $200$ words each, $10$ topics and $50$ different tokens.
We will randomly generate our generative model's parameters with uniform <em>topic_proba</em> and <em>word_proba</em> coming from a [Dirichlet](https://en.wikipedia.org/wiki/Dirichlet_distribution) distribution with parameter $\alpha=0.5$.

### Question 6:
- How can we select the correct number of topics for a dataset ?
- Implement this idea in the following cell.

In [ ]:
n_train_docs = 100
n_valid_docs = 1000
doc_length = 200
n_topics = 10
n_tokens = 50
alpha = 0.5


def gen_bigger_corpus(
    n_train_docs, n_valid_docs, doc_length,
    n_topics, n_tokens, alpha
):
    topic_proba = np.array([1./n_topics] * n_topics)
    word_proba = np.random.dirichlet([alpha] * n_tokens, size=n_topics)

    corpus, topics = gen_corpus(n_train_docs, n_topics, n_tokens, doc_length, topic_proba, word_proba)
    valid_corpus, valid_topics = gen_corpus(n_valid_docs, n_topics, n_tokens, doc_length, topic_proba, word_proba)
    return topic_proba, word_proba, corpus, valid_corpus

# Implement your method here.
# Use the previous function to generate your dataset

# Application to Text
This model is well suited to represent text. In the following, we will apply our EM algorithm to discover topics in the _newsgroup 20_ corpus.

As is standard in Natural Language Processing, we will first apply a few pre-processing steps to the corpus as described [here](https://towardsdatascience.com/nlp-extracting-the-main-topics-from-your-dataset-using-lda-in-minutes-21486f5aa925).

Make sure you have installed the following libraries into your conda environment : 
- gensim
- nltk
- scikit-learn

First, we download the dataset, and wordnet.

In [ ]:
import gensim
import nltk

from nltk.stem import WordNetLemmatizer, SnowballStemmer
from sklearn.datasets import fetch_20newsgroups

newsgroups = {
    'train': fetch_20newsgroups(subset='train', shuffle = True),
    'test': fetch_20newsgroups(subset='test', shuffle = True)
}
nltk.download('wordnet')


###  Preprocessing :
We use the same pre-processing as in the blog-post :
- **Tokenization** : Split the text into sentences and the sentences into words. Lowercase the words and remove punctuation.
- Words that have fewer than 3 characters are removed.
- All **stopwords** are removed.
- Words are **lemmatized** - third person is changed to first person, all verbs are changed into present tense
- Words are **stemmed** - words are reduced to their root form.

In [ ]:
stemmer = SnowballStemmer("english")

def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))

# Tokenize and lemmatize
def preprocess(text):
    result=[]
    for token in gensim.utils.simple_preprocess(text) :
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
            
    return result

preprocess("The quick brown fox jumped over the lazy dogs.")

### Dictionary making:
Each unique element in the output of _preprocess_ is a token. We need to convert these into unique ids to run our algorithm. This is called building a **dictionary**. The library _gensim_ has a helper function to help us do this.

### Question 7:
- Why is all this preprocessing useful ?

In [ ]:
processed_docs = {s: [] for s in ['train', 'test']}
# This may take a while. We are processing the entire dataset.
for s in ['train', 'test']:
    for doc in newsgroups[s].data:
        processed_docs[s].append(preprocess(doc))
    
dictionary = gensim.corpora.Dictionary(processed_docs['train'])
print(f"{len(dictionary)} unique tokens before filtering")
dictionary.filter_extremes(no_below=15, no_above=0.1)
print(f"{len(dictionary)} unique tokens after filtering")

In [ ]:
# transform our corpus of text to lists of counts of tokens
bow_corpus = {
    s: [dictionary.doc2bow(doc) for doc in processed_docs[s]]
    for s in ['train', 'test']
}

n_tokens = len(dictionary)
text_corpus = {'train': None, 'test': None}
for s in text_corpus.keys():
    n_docs = len(processed_docs[s])
    text_corpus[s] = np.zeros((n_docs, n_tokens), dtype='int32')
    for d, bow in enumerate(bow_corpus[s]):
        for (id, value) in bow:
            text_corpus[s][d, id] = value

### Model learning
Now that the dataset is ready run EM on it, using $5$ topics. Since the dataset is bigger, this could take some time.

In [ ]:
# New E_step using bag of words rather than dense count arrays
def E_step_sparse(corpus, learned_topic_proba, learned_word_proba):
    n_topics = learned_topic_proba.shape[0]
    res = np.zeros((len(corpus), n_topics))
    for d, tuples in enumerate(corpus):
        maxi = np.finfo('float64').min
        for topic in range(n_topics):
            cur = np.log(learned_topic_proba[topic])
            for tok_id, v in tuples:
                tok_prob = learned_word_proba[topic, tok_id]
                cur += np.log(tok_prob) * v
            res[d, topic] = cur
            maxi = max(maxi, cur)
        total = 0
        for topic in range(n_topics):
            total += np.exp(res[d, topic] - maxi)
            res[d, topic] = np.exp(res[d, topic] - maxi)
        res[d, :] /= total

    return res

n_iter = 10
n_topics = 5
n_docs = len(bow_corpus['train'])



learned_topic_proba = np.ones((n_topics,)) / n_topics
learned_word_proba = np.empty((n_topics, n_tokens), dtype='float32')
# initialize topic probabilities with random documents
permutation = np.random.permutation(n_docs)
batch_size = n_docs // n_topics
for t in range(n_topics):
    stats = np.sum(text_corpus['train'][permutation[t * batch_size: (t+1) * batch_size]], axis=0)
    learned_word_proba[t] = stats / np.sum(stats)

train_ll = []
valid_ll = []
for i in range(n_iter):
    # E-step
    doc_topic_probas = E_step_sparse(bow_corpus['train'], learned_topic_proba, learned_word_proba)

    # E-step for valid
    valid_topic_probas = E_step_sparse(bow_corpus['test'], learned_topic_proba, learned_word_proba)

    # M-step
    learned_topic_proba, learned_word_proba = M_step(text_corpus['train'], doc_topic_probas)

    train_ll.append(log_likelihood(text_corpus['train'], learned_word_proba, learned_topic_proba))    
    valid_ll.append(log_likelihood(text_corpus['test'], learned_word_proba, learned_topic_proba))
    
plt.figure(dpi=150)
p, = plt.plot(range(len(train_ll)), train_ll, '-', label=f"{n_topics} topics")
plt.plot(range(len(valid_ll)), valid_ll, '--', c=p.get_color())
    
plt.xlabel("EM-iteration")
plt.ylabel("Data log-likelihood")
plt.legend(bbox_to_anchor=(1,1), loc=2)
plt.show()


### Visualizing the topics

Now that we've run EM, we can print the most frequent terms associated to each topics.

In [ ]:
print(learned_topic_proba)
for t in range(n_topics):
    most_frequent = np.argsort(learned_word_proba[t])[-10:]
    print(f"------- Topic {t} ------")
    for w in most_frequent:
        print(f"\t -- {dictionary[int(w)]}")

### Perplexity
The [perplexity](https://en.wikipedia.org/wiki/Perplexity) of a model $q$ on a test sample $(x_s)$ is given by :
$$2^{-\frac{1}{S}\sum_{s=1}^S\log_2(q(x_s))}$$

### Question 8 :
Relate this quantity to one of the quantities we've used in this practical session.



### Latent Dirichlet Allocation
A [more complicated graphical model](https://en.wikipedia.org/wiki/Latent_Dirichlet_allocation) allows documents to come from mixtures of topics. The library _gensim_ can learn this model for us.

In [ ]:
lda_model =  gensim.models.LdaMulticore(
    bow_corpus['train'], num_topics = 5, id2word = dictionary, passes = 10, workers = 1
)

In [ ]:
for idx, topic in lda_model.print_topics(-1):
    print("Topic: {} \nWords: {}".format(idx, topic ))
    print("\n")

The Latent Dirichlet Allocation model is linked to the following generative process (multinomial PCA):
- Each document now has a mixture of topic $\theta_i$
- For each word, first draw a topic from $\theta_i$, then draw a word from this topic.

### Bonus Questions :
- Write the conditional probability distributions of the topic and the words.
- Derive the E and M step for this model.
- In the code implemented for question 6, what happens when we increase the number of train documents ? Comment.
- What can you notice regarding the computations done in the E-M algorithm for this practical session ? How would you implement this algorithm in a distributed fashion ?